## Final figures

In [3]:
%matplotlib widget
import pyphysics as phys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.axes as mplaxes
# Color map
plt.rcParams["image.cmap"] = "managua_r"
# plt.rcParams["savefig.bbox"] = "standard"

import util as u
from importlib import reload

### Track reconstruction in merger

In [38]:
reload(u)
ev = u.DataInterface("../Events/fine_after.root")
ev.change_id({0:0, 1:2, 2:1})

fig = u.init_figure("event rec", figsize=(5.5, 3.5))
gs = mpl.gridspec.GridSpec(1, 2, width_ratios=[0.8, 0.2])
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
# for ax in [ax1]:
#     ax.set_box_aspect(1)

# Event
plt.sca(ax1)
ev.draw(True)
ev.draw_rp(color="dodgerblue", mfc="dodgerblue", markersize=12, zorder=5)
ev.draw_line(1, lw=0.75)
# Annotate RP
ax1.annotate("RP", xy=(ev.rp[0], ev.rp[1] + 3), xytext=(ev.rp[0], ev.rp[1] + 30), fontsize=14,
             va="center", ha="center",
             arrowprops=dict(arrowstyle="-"))

# Silicons
plt.sca(ax2)
ax2.set_ylim([0, 1])
ax2.set_xlim([0, 0.2])
ax2.grid(which="both")
# ax2.axhline(y=0.5, color="blue")
# ax2.get_yaxis().set_visible(False)

def mirror(plot, x, y):
    ynew = [1 - e for e in y]
    plt.plot([x for _ in range(len(y))], ynew, color=plot.get_color(), ls=plot.get_linestyle(), lw=plot.get_linewidth())

#f0
x0 = 0.10
y0 = [0.1, 0.45]
f0, = plt.plot([x0, x0], y0, color="darkorange", lw=4)
mirror(f0, x0, y0)

#f1
x1 = 0.15
y1 = [0.1, 0.45]
f1, = plt.plot([x1, x1], y1, color="darkkhaki", lw=4)
mirror(f1, x1, y1)

#Line in active region
def transform(xmin, xmax):
    xd = np.linspace(xmin, xmax, 2)
    yd = [ev.eval_line(0, x) for x in xd]
    ac = np.array([(ax1.transData + ax1.transAxes.inverted()).transform([x, y]) for x,y in zip(xd, yd)])
    # print("ac: ", ac)
    fc = np.array([(ax1.transAxes + fig.transFigure.inverted()).transform([x, y]) for x,y in ac])
    # print("fc: ", fc)
    return fc
# Line in active region
lactive = transform(ev.rp[0], 128)
l0 = mpl.lines.Line2D(lactive[:,0], lactive[:,1], color ="darkorange", lw=1)
l0.set_transform(fig.transFigure)
fig.add_artist(l0)
# Line in distance to silicons
ltosil = transform(128, 160)
l1 = mpl.lines.Line2D(ltosil[:,0], ltosil[:,1], color ="darkorange", lw=1, ls=":")
l1.set_transform(fig.transFigure)
fig.add_artist(l1)

# Annotate charge average and TL
ax1.annotate(r"\noindent$\Delta\mathrm{E}_{\mathrm{gas}} = \sum_{i}{\mathrm{Q}_{i}} / \mathrm{TL}_{\mathrm{pad}}$\\ \\$\mathrm{TL} = \mathrm{TL}_{\mathrm{pad}} + \mathrm{TL}_{\mathrm{sil}}$", xy=(86, 56), xytext=(45, 30), fontsize=14,
            arrowprops=dict(arrowstyle="-"))

# Distance to silicons
ax1.annotate("", xy=(0.685, 0.75), xytext=(0.831,0.75),
            xycoords=fig.transFigure, textcoords=fig.transFigure,
            arrowprops=dict(arrowstyle="<->"))
ax1.annotate(r"d $\sim$ 10 cm", xy=(0.76, 0.78), ha="center",
            xycoords=fig.transFigure, fontsize=10)
ax2.set_axis_off()

# Theta
ax1.annotate("", xy=(98, 64.5), xytext=(97, 52),
            arrowprops=dict(arrowstyle="-", connectionstyle="arc3,rad=.3"))
ax1.annotate(r"$\theta$", xy=(102, 55), fontsize=14)

# DeltaE sil
ax1.annotate(r"E$_0$ E$_1$", xy=(0.815, 0.12), xycoords=fig.transFigure, fontsize=14)

# Silicon point
ax2.plot(0.1, 0.235, color="red", mfc="red", marker="*", ms=12, zorder=4)
ax1.annotate("SP", xy=(0.815, 0.305), xycoords=fig.transFigure, xytext=(0.76, 0.42),
            fontsize=14, ha="center", va="center", arrowprops=dict(arrowstyle="-"))

# DO NOT CALL THIS OTHERWISE THE POSITIONS WILL CHANGE
# plt.tight_layout()
plt.show()
plt.savefig("./Outputs/merger.pdf")

### 2D Histograms